In [1]:
import numpy as np
from queue import PriorityQueue as queue
from collections import defaultdict
from faker import Faker
from tabulate import tabulate
from collections import deque

fake = Faker()

## Draw Simulation Methods

In [2]:
def init_players(num_players):
    """
    Create a collection of players in the form [player_id, player_name, player_ranking]
    Arguments:
        num_players: Number of players to add to the draw
    Returns:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
    """
    players = []
    id_to_name, id_to_rank = {}, {}
    for n in range(num_players):
        p_id, p_name, p_rank = np.random.randint(1000000, 9999999), fake.name(), np.random.randint(1600, 2000)
        players.append([p_id, p_name, p_rank])
        id_to_name[p_id] = p_name
        id_to_rank[p_id] = p_rank
    return players, id_to_name, id_to_rank

In [3]:
def init_pairings(players):
    """
    Randomly create results for the first round
    Arguments:
        players: (n x 3) numpy array containing [player_id, player_name, player_ranking]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    for n in range(0, len(players), 2):
        a, b = players[n][0], players[n+1][0]
        result = [a, b, np.random.choice([a, b, None], 1, p=(0.45, 0.45, 0.1))[0]]
        results.append(result)
    return results

In [4]:
def run_round(pairings, p_draw=0):
    """
    Randomly create results for a round
    Arguments:
        pairings: List of pairings for the round to be simulated
            (n x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    p_win = (1 - p_draw) / 2
    for pairing in pairings:
        a, b = pairing[0][1], pairing[1][1]
        result = [a, b, np.random.choice([a, b, None], 1, p=(p_win, p_win, p_draw))[0]]
        results.append(result)
        
    return results

## Draw Calculation Methods

In [5]:
def create_draw(players, pairs):
    """
    Create draw for the round
    Agruments:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
        pairs: List of pairings for all rounds to date
            (m x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        splits: List of index points where each points bracket finishes
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
        pairings: The list form of pairing allocations
            (n / 2 x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    """
    n_players = len(players)

    rankings = defaultdict(int)
    if len(pairs) > 0:
        for pair in pairs:
            rankings[pair[0]] += (pair[0] == pair[2])
            rankings[pair[1]] += (pair[1] == pair[2])
    else:
        for player in players:
            rankings[player[0]] = 0

    standings = np.transpose([list(rankings.keys()), list(rankings.values())])
    standings = np.array(sorted(standings, key=lambda x: x[1], reverse=True))
    
    scores, splits  = get_scores(standings, pairs)
    alloc, pairings = get_allocations(standings, scores)
    
    return standings, splits, scores, alloc, pairings

In [6]:
def get_scores(standings, pairs):
    """
    Assign scores to each possible pairing
    A higher score means that the pairing is more strongly preferred
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        pairs: List of pairings for all rounds to date
            (m x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
        splits: List of index points where each points bracket finishes
    """
    n_players = len(standings)
    ix = {player_id: pos for pos, player_id in enumerate(standings[:,0])}
    splits = [n for n in range(n_players) if standings[n,1] != standings[n-1,1]]
    
    # Calculate scores for pairings
    scores = np.full((n_players, n_players), -100)
    
    split_1, split_2 = 0, 0
    for split in splits + [n_players]:
        # Assign points for players in same bracket
        scores[split_1:split, split_1:split] = 4
        # Assign points for pull-up bracket
        if split_1 %2 == 1:
            scores[split_2:split_1, split_1:split] = 1
            scores[split_1:split, split_2:split_1] = 1
        split_1, split_2 = split, split_1
    
    # Reset self vs self to -inf
    np.fill_diagonal(scores, -100)

    # Subtract points for prior matchup
    for pair in pairs:
        a, b = ix[pair[0]], ix[pair[1]]
        scores[a,b] -= 1
        scores[b,a] -= 1
        
    return scores, splits

In [7]:
def swap_pairs(pair_1, pair_2, alloc):
    """
    Swap players between two pairings
    Original pairing of (a,b) and (c,d) becomes (a,c) and (b,d)
    Arguments:
        pair_1: Tuple containing first pair to swap (a,b)
        pair_2: Tuple containing first pair to swap (c,d)
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
    Returns:
        alloc: The matrix form pairing allocations with [(a,b), (c,d)] -> [(a,c), (b,d)]
            (n x n) matrix of integers        
    """
    a, b = pair_1
    c, d = pair_2

    alloc[a, b] = 0
    alloc[b, a] = 0
    alloc[c, d] = 0
    alloc[d, c] = 0
    alloc[a, c] = 1
    alloc[c, a] = 1
    alloc[b, d] = 1
    alloc[d, b] = 1
    
    return alloc

In [8]:
def get_allocations(standings, scores):
    """
    Allocate player pairings
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
    Returns:
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
        pairs: The list form of pairing allocations
            (n / 2 x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    """
    n_players = len(standings)
    ix = {pos: player_id for pos, player_id in enumerate(standings[:,0])}
    alloc = np.zeros((n_players, n_players), dtype=int)

    # Construct initial draw
    for _ in range(int(n_players / 2)):
        mask = (1 - alloc.sum(axis=1))
        # Choose Player A
        p = scores.max() - scores.mean(axis=1)
        p = p * mask
        a = p.argmax()
        # Choose Player B
        p = np.exp(scores[a])
        p = np.maximum(p * mask, 0)
        b = np.random.choice(range(n_players),p=p/p.sum())
        
        alloc[a, b] = 1
        alloc[b, a] = 1

    # Run Metropolis-Hastings to iteratively improve draw
    prob = np.exp(scores)
    prob /= prob.sum(axis=1, keepdims=True)

    score = (alloc*scores).sum()/len(prob)
    score_q = deque([-100, -100, score], 3)
    best_alloc = alloc.copy()
    while score != min(score_q):
        for a, p in enumerate(prob):
            b = alloc[a].argmax()
            c = np.random.choice(range(len(p)),p=p)
            d = alloc[c].argmax()
            p_change = prob[a,c]*prob[b,d]/(prob[a,b]*prob[c,d] + prob[a,c]*prob[b,d])
            if np.random.random() < p_change:
                alloc = swap_pairs((a,b), (c,d), alloc)

        score = (alloc*scores).sum()/len(prob)
        if score > max(score_q):
            best_alloc = alloc.copy()
        score_q.append(score)
    
    alloc = best_alloc
    del(best_alloc)
    
    # Create list form of pairings
    alloc_ = alloc * np.tri(*alloc.shape)

    pairs = []
    for n in range(n_players):
        if alloc_[n].max() == 1:
            a, b = n, alloc_[n].argmax()
            pairs.append([(a, ix[a]), (b, ix[b])])

    return alloc, pairs

## Draw Printing Methods

In [9]:
def print_standings(standings, id_to_name):
    """
    Print standings for the round in table form
    """
    table = []
    for rank, player in enumerate(standings):
        table.append([int(player[0]), id_to_name[player[0]], player[1], rank+1])
    print('\n', tabulate(table, headers=['ID', 'Name', 'Wins', 'Rank']), '\n')
    

def print_pairings(pairs, id_to_name, scores):
    """
    Print pairings for the round in table form
    """
    table = []
    for table_num, pair in enumerate(pairs):
        b, a = pair[0], pair[1]
        score = scores[b[0], a[0]]
        row = [table_num+1, id_to_name[a[1]], int(a[1]), int(a[0])+1, id_to_name[b[1]], int(b[1]), int(b[0])+1, score]
        table.append(row)
    headers = ['Table #', 'Player 1 - Name', 'ID', 'Rank', 'Player 2 - Name', 'ID', 'Rank', 'GOF']
    print('\n', tabulate(table, headers=headers), '\n')

## Run a sample Tournament
Create a list of players then run through a multi-round tournament and print the results
Each round prints:
- Goodness of fit score: Score for the pairings allocated in the round. The maximum possible score is 4. If a number < 0 comes out, something has probably gone wrong
- Standings: A table of standings of the players (based only on # of wins, no accounting for draws, op wins etc)
- Pairings: A table of pairings for the following round (including player name and standing so you can check if players are being paired with others nearby)

To show all the standings and pairings at once, make sure to select the cell, click on the "Cell" menu, select -> "Current Outputs" and then select "Toggle Scrolling"

The entire history of "standings", "splits", "scores", "allocations", and "pairs" is recorded in the lists *standings_h, splits_h, scores_h, alloc_h, pairs_h*
    
Note: Early stage demo only. This does not support:
- Odd numbers of players and byes
- Allocating pods
- Iterative sampling methods to improve draw quality
- etc

In [10]:
NUMBER_OF_PLAYERS = 26
NUMBER_OF_ROUNDS  = 6
PROBABILITY_OF_DRAW = 0.0001

In [11]:
# Create players
players, id_to_name, id_to_rank = init_players(NUMBER_OF_PLAYERS)
pairings = init_pairings(players)

#players

In [12]:
results = []
standings_h, splits_h, scores_h, alloc_h, pairs_h = [], [], [], [], []

for rnd in range(NUMBER_OF_ROUNDS):
    standings, splits, scores, alloc, pairs = create_draw(players, results)
    for collection, value in zip([standings_h, splits_h, scores_h, alloc_h, pairs_h],
                                 [standings,   splits,   scores,   alloc,   pairs]):
        collection.append(value)
    print('----- Round', rnd + 1, '-----', )
    print('Goodness of Fit:', (scores * alloc).sum() / NUMBER_OF_PLAYERS)
    print_standings(standings, id_to_name, )
    print_pairings(pairs, id_to_name, scores)
    result = run_round(pairs, p_draw=PROBABILITY_OF_DRAW)
    results += result

----- Round 1 -----
Goodness of Fit: 4.0

      ID  Name                       Wins    Rank
-------  -----------------------  ------  ------
6647878  Jennifer Hayes                0       1
2374253  Dawn Martin                   0       2
3317725  Caleb Reyes                   0       3
1698462  Rebecca Hamilton              0       4
1240654  Timothy Hess                  0       5
4062633  Sandra Yang                   0       6
2708757  April Lee                     0       7
3850883  Mr. Peter Richardson II       0       8
6724236  Christopher Scott             0       9
1381997  Kristen Garrison              0      10
4495722  Michael Lee                   0      11
6009545  Jeffrey Wright                0      12
3984681  Danielle Navarro              0      13
4311220  Pamela Lowe                   0      14
2591711  Denise Ward                   0      15
3643808  William Powers                0      16
4249005  Bryan Walsh                   0      17
5585646  Jesus Mcdonald   